In [1]:
# Parameters
data_url = "https://osf.io/5ubn8/"


In [2]:
import endomill
from nbmetalog import nbmetalog as nbm


In [3]:
nbm.print_metadata()


context: ci
hostname: 87b688f1e3b8
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 3
nbname: phylogeny_simulation_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_simulation_mill.ipynb
revision: null
session: 1b71db11-7bde-4136-bd78-bd037bdc0593
timestamp: 2022-03-06T07:12:11Z00:00


IPython==7.16.1
keyname==0.4.1
yaml==5.3.1
endomill==0.1.2
nbmetalog==0.2.6
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.3.0
json==2.0.9
ipykernel==5.5.3


In [4]:
endomill.instantiate_over(
    parameter_packs=[
        {'data_url': data_url}
        for data_url in [
            'https://osf.io/cz9fk/',
            'https://osf.io/ydxt7/',
            'https://osf.io/8ycq7/',
            'https://osf.io/5ubn8/',
        ]
    ],
)


detected executing.endomill.ipynb file
skipping instantiate_over


In [5]:
# define papermill parameters
data_url: str


In [6]:
import anytree
from bs4 import BeautifulSoup
from collections import defaultdict
from hstrat import hstrat
from interval_search import doubling_search
from iterpop import iterpop as ip
import itertools as it
from keyname import keyname as kn
import opytional as opyt
import pandas as pd
import random
import sys
from tqdm import tqdm
from urllib import request


In [7]:
random.seed(1) # ensure reproducibility
sys.setrecursionlimit(100000) # anytree exceeds default recursion limit on our data


In [8]:
nbm.print_metadata()


context: ci
hostname: 87b688f1e3b8
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 8
nbname: phylogeny_simulation_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_simulation_mill.ipynb
revision: null
session: 1b71db11-7bde-4136-bd78-bd037bdc0593
timestamp: 2022-03-06T07:12:12Z00:00




IPython==7.16.1
endomill==0.1.2
ipykernel==5.5.3
ipython_genutils==0.2.0
json==2.0.9
keyname==0.4.1
logging==0.5.1.2
nbmetalog==0.2.6
re==2.2.1
yaml==5.3.1
zmq==22.3.0
anytree==2.8.0
hstrat==0.2.0
iterpop==0.3.4
opytional==0.1.0
pandas==1.1.2


In [9]:
html = request.urlopen(data_url).read().decode('utf8')

soup = BeautifulSoup(html, 'html.parser')
title = soup.find('title')

data_filename = title.string.split()[-1]
print(data_filename)


nk_tournamentselection_seed140_pop100_mut.01_snapshot_5000.csv


In [10]:
endomill.add_instance_outpath(
    f'a=phylogeny_simulation+source={data_filename}.endomill.ipynb'
)


# Retrieve Target Phylogeny from OSF


In [11]:
target_phylogeny_df = pd.read_csv(
    f'{data_url}/download',
)

nbm.print_dataframe_synopsis(target_phylogeny_df)


digest: 4c251759926792904321cbc4ed9dfae37e26ff5251f1f60570325be9fcfae704
manifest:
  ancestor_list: '    9#   ex., [250]'
  depth: '            9#   ex., 8'
  destruction_time: ' 7#   ex., inf'
  id: '               17#  ex., 47645'
  num_offspring: '    3#   ex., 0'
  num_orgs: '         3#   ex., 1'
  origin_time: '      10#  ex., 5000'
  phenotype: '        15#  ex., [ 0.897281 0.627867 0.842173 0.953767 0.615421 0.849175
    0.795495 0.655207 0.630174 0.939279 0.457317 0.102534 0.960229 0.780958 0.248017
    0.843595 0.791117 0.595291 0.68697 0.513536 ]'
  tot_orgs: '         9#   ex., 1'
  total_offspring: '  3#   ex., 0'
num cols: 10
num cols all na: 0
num cols any na: 0
num na: 0
num rows: 17
num rows all na: 0
num rows any na: 0
size: 6K



# Create a Tree with Target Phylogeny Structure


In [12]:
# map id to anytree node
nodes = defaultdict(anytree.AnyNode)
for __, row in target_phylogeny_df.iterrows():
    node = nodes[row['id']]
    node.id = row['id']
    node.origin_time = row['origin_time']
    if 'NONE' not in row['ancestor_list']:
        ancestor_id = ip.popsingleton(
            eval(row['ancestor_list'])
        )
        node.parent = nodes[ancestor_id]

roots = {node.root for node in nodes.values()}


In [13]:
root = ip.popsingleton(roots)
root.height


8

# Pick Parameters for Hereditary Stratigraphic Columns


In [14]:
def make_conditions(num_generations: int) -> pd.DataFrame:
    res = []
    for condemner_factory, target_column_bits, differentia_bit_width in it.product(
        [
            hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution,
            hstrat.StratumRetentionCondemnerRecencyProportionalResolution,
        ],
        [
            64,
            64 * 8,
            64 * 16,
        ],
        [
            1,
            8,
            64,
        ],
    ):
        policy_param = doubling_search(
            lambda x: \
                condemner_factory(x + 1).CalcNumStrataRetainedExact(num_generations)
                * differentia_bit_width > target_column_bits or x >= num_generations,
            {
                hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution: 1,
                hstrat.StratumRetentionCondemnerRecencyProportionalResolution: 0,
            }[condemner_factory],
        )

        actual_column_strata = condemner_factory(policy_param).CalcNumStrataRetainedExact(num_generations)
        actual_column_bits = actual_column_strata * differentia_bit_width

        res.append({
            'Retention Policy' : condemner_factory.__name__[25:],
            'Differentia Bit Width' : differentia_bit_width,
            'Retention Policy Resolution Parameter' : policy_param,
            'Target Retained Bits' : target_column_bits,
            'Actual Retained Bits' : actual_column_bits,
            'Retained Bits Error' : actual_column_bits - target_column_bits,
            'Actual Retained Strata' : actual_column_strata,
            'condemner' : condemner_factory(policy_param),
        })
    return pd.DataFrame.from_records(res)


In [15]:
conditions_df = make_conditions(root.height + 1)
conditions_df.drop('condemner', axis=1)


,Retention Policy,Differentia Bit Width,Retention Policy Resolution Parameter,Target Retained Bits,Actual Retained Bits,Retained Bits Error,Actual Retained Strata
0,TaperedDepthProportionalResolution,1,9,64,9,-55,9
1,TaperedDepthProportionalResolution,8,4,64,64,0,8
2,TaperedDepthProportionalResolution,64,1,64,128,64,2
3,TaperedDepthProportionalResolution,1,9,512,9,-503,9
4,TaperedDepthProportionalResolution,8,9,512,72,-440,9
5,TaperedDepthProportionalResolution,64,4,512,512,0,8
6,TaperedDepthProportionalResolution,1,9,1024,9,-1015,9
7,TaperedDepthProportionalResolution,8,9,1024,72,-952,9
8,TaperedDepthProportionalResolution,64,9,1024,576,-448,9
9,RecencyProportionalResolution,1,9,64,9,-55,9


# Set Up Ancestor Column


In [16]:
bundle = hstrat.HereditaryStratigraphicColumnBundle({
    kn.pack({
        'differentia' : row['Differentia Bit Width'],
        'policy' : row['Retention Policy'],
        'resolution' : row['Retention Policy Resolution Parameter'],
        'target_bits' : row['Target Retained Bits'],
        'actual_bits' : row['Actual Retained Bits'],
        'bits_error' : row['Retained Bits Error'],
        'actual_strata' : row['Actual Retained Strata'],
    }) \
        : hstrat.HereditaryStratigraphicColumn(
            stratum_differentia_bit_width=row['Differentia Bit Width'],
            stratum_retention_condemner=row['condemner'],
    )
    for __, row in conditions_df.iterrows()
})


# Simulate Inheritance of Ancestor Column Down Phylogenetic Tree


In [17]:
root = ip.popsingleton(roots)
root.hstrat_column = bundle

for node in anytree.LevelOrderIter(ip.popsingleton(roots)):
    parent = node.parent
    if parent is not None:
        node.hstrat_column = parent.hstrat_column.Clone()
        for __ in range(node.origin_time - parent.origin_time): node.hstrat_column.DepositStratum()


# Extract Pairwise MRCA Estimates for Extant Organisms


In [18]:
res = []
for extant1, extant2 in tqdm([*it.product(root.leaves, root.leaves)]):
    if extant1 != extant2:
        bounds = extant1.hstrat_column.CalcRankOfMrcaBoundsWith(extant2.hstrat_column)
        for impl in extant1.hstrat_column:
            res.append({
                'Column Configuration' \
                    : impl,
                'Differentia Bit Width' \
                    : kn.unpack(impl)['differentia'],
                'Stratum Retention Policy' \
                    : kn.unpack(impl)['policy'],
                'Stratum Retention Policy Resolution Parameter' \
                    : kn.unpack(impl)['resolution'],
                'Stratigraphic Column Actual Retained Bits' \
                    : kn.unpack(impl)['actual_bits'],
                'Stratigraphic Column Target Retained Bits' \
                    : kn.unpack(impl)['target_bits'],
                'Stratigraphic Column Retained Bits Error' \
                    : kn.unpack(impl)['bits_error'],
                'Stratigraphic Column Actual Num Retained Strata' \
                    : kn.unpack(impl)['actual_strata'],
                'Taxon Compared From' \
                    : extant1.id,
                'Taxon Compared To' \
                    : extant2.id,
                'Generation of Taxon Compared From' \
                    : extant1.hstrat_column.GetNumStrataDeposited(),
                'Generation of Taxon Compared To' \
                    : extant2.hstrat_column.GetNumStrataDeposited(),
                'Generation Of MRCA Lower Bound (inclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[0],
                    ),
                'Generation Of MRCA Upper Bound (exclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[1],
                    ),
                'MRCA Bound Confidence' \
                    : extant1.hstrat_column[impl].CalcRankOfMrcaBoundsWithProvidedConfidenceLevel(),
                'Rank of Earliest Detectable Mrca With' \
                    : extant1.hstrat_column[impl].CalcRankOfEarliestDetectableMrcaWith(extant2.hstrat_column[impl]),
            })

res_df = pd.DataFrame.from_records(res)


  0%|          | 0/81 [00:00<?, ?it/s]

100%|██████████| 81/81 [00:00<00:00, 858.34it/s]

In [19]:
res_df


,Column Configuration,Differentia Bit Width,Stratum Retention Policy,Stratum Retention Policy Resolution Parameter,Stratigraphic Column Actual Retained Bits,Stratigraphic Column Target Retained Bits,Stratigraphic Column Retained Bits Error,Stratigraphic Column Actual Num Retained Strata,Taxon Compared From,Taxon Compared To,Generation of Taxon Compared From,Generation of Taxon Compared To,Generation Of MRCA Lower Bound (inclusive),Generation Of MRCA Upper Bound (exclusive),MRCA Bound Confidence,Rank of Earliest Detectable Mrca With
0,actual_bits=9+actual_strata=9+bits_error=-55+d...,1,TaperedDepthProportionalResolution,9,9,64,-55,9,47645,47644,5001,5001,NaN,NaN,0.968750,1536
1,actual_bits=64+actual_strata=8+bits_error=0+di...,8,TaperedDepthProportionalResolution,4,64,64,0,8,47645,47644,5001,5001,0.0,1024.0,0.996094,0
2,actual_bits=128+actual_strata=2+bits_error=64+...,64,TaperedDepthProportionalResolution,1,128,64,64,2,47645,47644,5001,5001,0.0,4096.0,1.000000,0
3,actual_bits=9+actual_strata=9+bits_error=-503+...,1,TaperedDepthProportionalResolution,9,9,512,-503,9,47645,47644,5001,5001,NaN,NaN,0.968750,1536
4,actual_bits=72+actual_strata=9+bits_error=-440...,8,TaperedDepthProportionalResolution,9,72,512,-440,9,47645,47644,5001,5001,0.0,512.0,0.996094,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,actual_bits=72+actual_strata=9+bits_error=-440...,8,RecencyProportionalResolution,9,72,512,-440,9,47633,47626,5000,5000,0.0,256.0,0.996094,0
1292,actual_bits=512+actual_strata=8+bits_error=0+d...,64,RecencyProportionalResolution,3,512,512,0,8,47633,47626,5000,5000,0.0,1024.0,1.000000,0
1293,actual_bits=9+actual_strata=9+bits_error=-1015...,1,RecencyProportionalResolution,9,9,1024,-1015,9,47633,47626,5000,5000,NaN,NaN,0.968750,1024
1294,actual_bits=72+actual_strata=9+bits_error=-952...,8,RecencyProportionalResolution,9,72,1024,-952,9,47633,47626,5000,5000,0.0,256.0,0.996094,0


In [20]:
nbm.print_dataframe_synopsis(res_df)


digest: 052934ba2ea68141ab8ceaa44cde41546953656aeb66e6814458f7a91cce81f4
manifest:
  Column Configuration: '                            18#        ex., actual_bits=64+actual_strata=8+bits_error=0+differentia=8+policy=TaperedDepthProportionalResolution+resolution=4+target_bits=64'
  Differentia Bit Width: '                           3#         ex., 8'
  Generation Of MRCA Lower Bound (inclusive): '      5#,408na   ex., 0.0'
  Generation Of MRCA Upper Bound (exclusive): '      10#,408na  ex., 1024.0'
  Generation of Taxon Compared From: '               2#         ex., 5001'
  Generation of Taxon Compared To: '                 2#         ex., 5001'
  MRCA Bound Confidence: '                           3#         ex., 0.99609375'
  Rank of Earliest Detectable Mrca With: '           3#         ex., 0'
  Stratigraphic Column Actual Num Retained Strata: ' 3#         ex., 8'
  Stratigraphic Column Actual Retained Bits: '       6#         ex., 64'
  Stratigraphic Column Retained Bits Error: '   

# Save Pairwise MRCA Estimates to File


In [21]:
res_df.to_csv(
    f'a=pairwise_mrca_estimates+source={data_filename}.gz',
    compression='gzip',
)
